<a href="https://colab.research.google.com/github/mebalint/3rd_homework/blob/main/3rd_homework_Meszaros_Balint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Deep Learning 3rd homework

In [ ]:
# importing packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(123)

In [ ]:
# also importing packages, but for deep learning
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler

In [ ]:
# reading the data of March 2014 in
list_of_dfs = pd.read_html('https://freemeteo.hu/idojaras/budapest/elozmenyek/havi-elozmenyek/?gid=3054643&station=4276&month=3&year=2014&language=hungarian&country=hungary')

In [ ]:
# printing the data out
list_of_dfs[6].iloc[:,0:3]

,Dátum,Napi min hőmérséklet,Napi max hőmérséklet
0,2014.03.01.,0°C,12°C
1,2014.03.02.,4°C,11°C
2,2014.03.03.,4°C,7°C
3,2014.03.04.,4°C,10°C
4,2014.03.05.,4°C,12°C
5,2014.03.06.,6°C,14°C
6,2014.03.07.,6°C,10°C
7,2014.03.08.,2°C,15°C
8,2014.03.09.,0°C,13°C
9,2014.03.10.,-1°C,12°C


In [ ]:
# a function for scraping data for a given month and year
def scraping(year, month, html1='https://freemeteo.hu/idojaras/budapest/elozmenyek/havi-elozmenyek/?gid=3054643&station=4276&month=', html2='&year=', html3='&language=hungarian&country=hungary'):
    list_of_dfs = pd.read_html(html1+str(month)+html2+str(year)+html3)
    table=list_of_dfs[6].iloc[:,0:3]
    return table

In [ ]:
# data is from 2011 to 2020
years=range(2011,2021)
months=range(1,13)

In [ ]:
# making the dataframes for every year and month
dataframes=[]
for year in years:
  for month in months:
    dataframes.append(scraping(year, month))


In [ ]:
# it is not important, i have just checked if every dataframe has a good length
# the number of days of the months
days=[31,28,31,30,31,30,31,31,30,31,30,31]
# checking for every dataframe
for j in range(120):
  if len(dataframes[j])!=days[j%12]:
    print(j)

13
35
47
59
61
68
69
70
71
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
100
107
109
119


In [ ]:
# these deletings are because the data is nan
dataframes[10]=dataframes[10].drop(labels=19,axis=0)

In [ ]:
dataframes[57]=dataframes[57].drop(labels=15,axis=0)

In [ ]:
dataframes[62]=dataframes[62].drop(labels=12,axis=0)

In [ ]:
dataframes[107]=dataframes[107].drop(labels=4,axis=0)

In [ ]:
dataframes[119]=dataframes[119].drop(labels=[17,18],axis=0)

In [ ]:
# dataframes for min temperature 
dfsmin=[]
for i in range(120):
  # selecting the min column, and transforming it as a float
  minim=[float(data[0:-2]) for data in dataframes[i].iloc[:,1].values]
  # every 7 succesive temperature determines the 8th temperature in my model
  traindata=[minim[i:i+8] for i in range(len(minim)-7)]
  df=pd.DataFrame(traindata, columns=['x1','x2','x3','x4','x5','x6','x7','y'])
  dfsmin.append(df)


In [ ]:
# dataframes for max temperature 
dfsmax=[]
for i in range(120):
  # selecting the max column, and transforming is as a float
  maxim=[float(data[0:-2]) for data in dataframes[i].iloc[:,2].values]
  # every 7 succesive temperature determines the 8th temperature in my model
  traindata=[maxim[i:i+8] for i in range(len(maxim)-7)]
  df=pd.DataFrame(traindata, columns=['x1','x2','x3','x4','x5','x6','x7','y'])
  dfsmax.append(df)


In [ ]:
# concatenating the dataframes of min temperatures
trainmin=pd.concat(dfsmin)
trainmin


,x1,x2,x3,x4,x5,x6,x7,y
0,-5.0,-5.0,-11.0,-5.0,-6.0,-10.0,-1.0,-1.0
1,-5.0,-11.0,-5.0,-6.0,-10.0,-1.0,-1.0,-3.0
2,-11.0,-5.0,-6.0,-10.0,-1.0,-1.0,-3.0,-2.0
3,-5.0,-6.0,-10.0,-1.0,-1.0,-3.0,-2.0,3.0
4,-6.0,-10.0,-1.0,-1.0,-3.0,-2.0,3.0,0.0
...,...,...,...,...,...,...,...,...
13,2.0,1.0,3.0,2.0,3.0,3.0,3.0,5.0
14,1.0,3.0,2.0,3.0,3.0,3.0,5.0,5.0
15,3.0,2.0,3.0,3.0,3.0,5.0,5.0,2.0
16,2.0,3.0,3.0,3.0,5.0,5.0,2.0,-4.0


In [ ]:
# concatenating the dataframes of max temperatures
trainmax=pd.concat(dfsmax)
trainmax

,x1,x2,x3,x4,x5,x6,x7,y
0,-2.0,1.0,-1.0,-3.0,-4.0,1.0,3.0,4.0
1,1.0,-1.0,-3.0,-4.0,1.0,3.0,4.0,4.0
2,-1.0,-3.0,-4.0,1.0,3.0,4.0,4.0,4.0
3,-3.0,-4.0,1.0,3.0,4.0,4.0,4.0,5.0
4,-4.0,1.0,3.0,4.0,4.0,4.0,5.0,7.0
...,...,...,...,...,...,...,...,...
13,3.0,3.0,4.0,4.0,4.0,5.0,5.0,11.0
14,3.0,4.0,4.0,4.0,5.0,5.0,11.0,9.0
15,4.0,4.0,4.0,5.0,5.0,11.0,9.0,6.0
16,4.0,4.0,5.0,5.0,11.0,9.0,6.0,5.0


In [ ]:
# writing out train data for max temperatures
trainmax.to_csv('trainmax.csv',index=False)

In [ ]:
# writing out train data for min temperatures
trainmin.to_csv('trainmin.csv',index=False)

In [ ]:
# reading in the previous csv, just for simplicity
trainmax=pd.read_csv('trainmax.csv')

In [ ]:
# transforming dataset
max_dataset=trainmax.values.astype('float32')

In [ ]:
# the first 7 columns are xi's these are given for feedforwarding,
# and the 8th column is the wanted result (y) 

# making train, test, validation sets
test_split = 0.1
valid_split = 0.1

X = max_dataset[:,0:7]
Y = max_dataset[:,7]

v_index = int(X.shape[0]*(1-valid_split-test_split))
t_index = int(X.shape[0]*(1-test_split))

X_test = X[t_index:]
Y_test = Y[t_index:]
X_valid = X[v_index:t_index]
Y_valid = Y[v_index:t_index]
X = X[:v_index]
Y = Y[:v_index]

# standardization of data
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
# we are looking for the best model, and our patience is 10
early_stopping=EarlyStopping(patience=10, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

In [ ]:
# making the model, one hidden layer with 64 neurons, and sigmoid activation
model = Sequential()
model.add(Dense(units=64, input_dim=X.shape[1], activation='sigmoid'))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['acc'])

history=model.fit(X,Y,epochs=1000, 
                  verbose=2,
                  validation_data=(X_valid, Y_valid),
                  callbacks=[checkpointer, early_stopping])

Epoch 1/1000
57/57 - 1s - loss: 342.2574 - acc: 0.0234 - val_loss: 361.7103 - val_acc: 0.0179

Epoch 00001: val_loss improved from inf to 361.71027, saving model to weights.hdf5
Epoch 2/1000
57/57 - 0s - loss: 270.6031 - acc: 0.0212 - val_loss: 284.9358 - val_acc: 0.0223

Epoch 00002: val_loss improved from 361.71027 to 284.93582, saving model to weights.hdf5
Epoch 3/1000
57/57 - 0s - loss: 210.8067 - acc: 0.0251 - val_loss: 221.4747 - val_acc: 0.0089

Epoch 00003: val_loss improved from 284.93582 to 221.47470, saving model to weights.hdf5
Epoch 4/1000
57/57 - 0s - loss: 161.6841 - acc: 0.0256 - val_loss: 169.3178 - val_acc: 0.0089

Epoch 00004: val_loss improved from 221.47470 to 169.31778, saving model to weights.hdf5
Epoch 5/1000
57/57 - 0s - loss: 122.7103 - acc: 0.0251 - val_loss: 128.2363 - val_acc: 0.0089

Epoch 00005: val_loss improved from 169.31778 to 128.23625, saving model to weights.hdf5
Epoch 6/1000
57/57 - 0s - loss: 92.0486 - acc: 0.0251 - val_loss: 95.5802 - val_acc: 0

In [ ]:
# the mse of the test dataset and the prediction of the model
from sklearn.metrics import mean_squared_error
model = load_model('weights.hdf5')
preds = model.predict(X_test)
test_err = mean_squared_error(Y_test,preds)
print("\nTesterror: %f" % (test_err))


Testerror: 10.181460


In [ ]:
# this is for max dataset
# predicting the weather, given 7 temperatures as last_one_week,
# and when we are curious about weather as prediction time
# recursion function, the result is the input of the next calculating
def weather_prediction_1(last_one_week,prediction_time):
  if prediction_time>0:
    prediction=model.predict(scaler.transform(last_one_week))
    # making the next seven length sample
    next_last_week=np.concatenate((last_one_week,prediction), axis=1)
    next_last_week=[np.delete(next_last_week,0)]
    #print(next_last_week)
    # calculating the next prediction
    weather_prediction_1(next_last_week,prediction_time-1)
  else:
    print(last_one_week[0][-1])

In [ ]:
# an example for 8th of November
weather_prediction_1([[14,10,12,16,12,13,11]],1)

11.104191780090332


In [ ]:
# reading in the earlier csv, just for simplicity
trainmin=pd.read_csv('trainmin.csv')

In [ ]:
# transforming dataset for min temperatures
min_dataset=trainmin.values.astype('float32')

In [ ]:
# the first 7 columns are xi's these are given for feedforwarding,
# and the 8th column is the wanted result (y) 

# making train, test, validation sets
X_2 = min_dataset[:,0:7]
Y_2 = min_dataset[:,7]

v_index = int(X_2.shape[0]*(1-valid_split-test_split))
t_index = int(X_2.shape[0]*(1-test_split))

X_test_2 = X_2[t_index:]
Y_test_2 = Y_2[t_index:]
X_valid_2 = X_2[v_index:t_index]
Y_valid_2 = Y_2[v_index:t_index]
X_2 = X_2[:v_index]
Y_2 = Y_2[:v_index]

#scaler = MinMaxScaler(feature_range=(0, 1))
scaler_2 = StandardScaler().fit(X_2)
X_2 = scaler_2.transform(X_2)
X_valid_2 = scaler_2.transform(X_valid_2)
X_test_2 = scaler_2.transform(X_test_2)

In [ ]:
early_stopping_2=EarlyStopping(patience=10, verbose=1)
checkpointer_2=ModelCheckpoint(filepath='weights_2.hdf5', save_best_only=True, verbose=1)

In [ ]:
# making another model for min temperatures, one hidden layer with 64 neurons, and sigmoid activation
model_2 = Sequential()
model_2.add(Dense(units=64, input_dim=X_2.shape[1], activation='sigmoid'))
model_2.add(Dense(units=1, activation='linear'))

model_2.compile(loss='mse', optimizer='adam', metrics=['acc'])

history=model_2.fit(X_2,Y_2,epochs=1000, 
                  verbose=2,
                  validation_data=(X_valid_2, Y_valid_2),
                  callbacks=[checkpointer_2, early_stopping_2])

Epoch 1/1000
57/57 - 1s - loss: 78.1400 - acc: 0.0462 - val_loss: 78.4771 - val_acc: 0.0491

Epoch 00001: val_loss improved from inf to 78.47712, saving model to weights_2.hdf5
Epoch 2/1000
57/57 - 0s - loss: 56.4395 - acc: 0.0468 - val_loss: 54.2166 - val_acc: 0.0491

Epoch 00002: val_loss improved from 78.47712 to 54.21665, saving model to weights_2.hdf5
Epoch 3/1000
57/57 - 0s - loss: 40.6265 - acc: 0.0468 - val_loss: 37.2507 - val_acc: 0.0491

Epoch 00003: val_loss improved from 54.21665 to 37.25068, saving model to weights_2.hdf5
Epoch 4/1000
57/57 - 0s - loss: 29.7040 - acc: 0.0507 - val_loss: 26.2166 - val_acc: 0.0491

Epoch 00004: val_loss improved from 37.25068 to 26.21662, saving model to weights_2.hdf5
Epoch 5/1000
57/57 - 0s - loss: 22.8174 - acc: 0.0540 - val_loss: 19.6752 - val_acc: 0.0446

Epoch 00005: val_loss improved from 26.21662 to 19.67520, saving model to weights_2.hdf5
Epoch 6/1000
57/57 - 0s - loss: 18.6668 - acc: 0.0529 - val_loss: 15.9243 - val_acc: 0.0491

Ep

In [ ]:
# the mse of the test dataset and the prediction of the model for min temperatures
model_2 = load_model('weights_2.hdf5')
preds_2 = model_2.predict(X_test_2)
test_err_2 = mean_squared_error(Y_test_2,preds_2)
print("\nTesterror: %f" % (test_err_2))


Testerror: 7.434148


In [ ]:
# this is for min dataset
# predicting the weather, given 7 temperatures as last_one_week,
# and when we are curious about weather as prediction time
# recursion function, the result is the input of the next calculating
def weather_prediction_2(last_one_week,prediction_time):
  if prediction_time>0:
    prediction=model_2.predict(scaler_2.transform(last_one_week))
    next_last_week=np.concatenate((last_one_week,prediction), axis=1)
    next_last_week=[np.delete(next_last_week,0)]
    #print(next_last_week)
    weather_prediction_2(next_last_week,prediction_time-1)
  else:
    print(last_one_week[0][-1])

In [ ]:
# an example for 8th ov November
weather_prediction_2([[2,6,1,8,2,-1,7]],1)

6.248996734619141


In [ ]:
weather_prediction_2([[2,6,1,8,2,-1,7]],1)
weather_prediction_1([[14,10,12,16,12,13,11]],1)
#so for 8th of November:
print('Expected average temperature for 8th of November:')
np.mean([6.248996734619141,
11.104191780090332])

6.248996734619141
11.104191780090332
Expected average temperature for 8th of November:


8.676594257354736

In [ ]:
weather_prediction_2([[2,6,1,8,2,-1,7]],8)
weather_prediction_1([[14,10,12,16,12,13,11]],8)
#so for 15th of November:
print('Expected average temperature for 15th of November:')
np.mean([4.641397476196289,
9.447543144226074])

4.641397476196289
9.447543144226074
Expected average temperature for 15th of November:


7.044470310211182

In [ ]:
weather_prediction_2([[2,6,1,8,2,-1,7]],29)
weather_prediction_1([[14,10,12,16,12,13,11]],29)
#so for 6th of December:
print('Expected average temperature for 6th of December:')
np.mean([2.4397642612457275,
4.759843826293945])

2.4397642612457275
4.759843826293945
Expected average temperature for 6th of December:


3.5998040437698364